In [2]:
import numpy as np
from vod.configuration import KittiLocations
from vod.frame import FrameTransformMatrix, FrameDataLoader, transform_pcl
import k3d
from my_fuse_radar.utils import plot_moving_objs_bbox, read_kitti_label_file, is_point_in_box, get_transformed_bbox_corners

kitti_locations = KittiLocations(root_dir='/datasets/vod')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [53]:
# 1370帧有轿车
frame_number = 20
frame_data = FrameDataLoader(kitti_locations=kitti_locations, frame_number=str(frame_number).zfill(5))
transforms = FrameTransformMatrix(frame_data)

_, moving_objs = read_kitti_label_file(frame_data.raw_labels)

lidar_points_cf = transform_pcl(frame_data.lidar_data, transforms.t_camera_lidar)[:,:3]
radar_points = np.fromfile('./fused10_comp.bin', dtype=np.float32).reshape(-1, 7)
# radar_points = np.fromfile(f'/datasets/vod/radar/training/velodyne/{str(frame_number).zfill(5)}.bin', dtype=np.float32).reshape(-1, 7)[:,:3]

timestamp = radar_points[:,6]
radar_points_cf = transform_pcl(radar_points, transforms.t_camera_radar)[:,:3]

truck_obj = moving_objs[5]
truck_obj_modified = {'id': 7,'type': 'truck','movement': '1',
    'height': 2.9,  # 稍微变矮
    'width': 2.6,   # 稍微变窄
    'length': 6.7+0.2,  # 稍微变短
    'x': 5.046-0.3,     # 保持位置不变
    'y': 2.614,
    'z': 8.156+0.9,
    'rotation': -0.9906283141074347+0.02  # 向左转了一点（+0.1 rad）
}

selected_rp_cf_uncomp = []
selected_rp_cf_comp = []
for idx, point in enumerate(radar_points_cf):
    if is_point_in_box(point, truck_obj, transforms.t_lidar_camera, transforms.t_camera_lidar):
        if timestamp[idx] < -1: # 需要被补偿的点
            selected_rp_cf_uncomp.append(point)
        else: # 不需要补偿的点
            selected_rp_cf_comp.append(point)

selected_rp_cf_uncomp = np.array(selected_rp_cf_uncomp)
selected_rp_cf_comp = np.array(selected_rp_cf_comp)

In [54]:
plot = k3d.plot(camera_auto_fit=False, axes_helper=True, grid_visible=False, antialias=True)
plot.renderer_pixel_ratio = 3.0

plot_moving_objs_bbox(plot, [truck_obj, truck_obj_modified], transforms.t_lidar_camera, transforms.t_camera_lidar)
plot_moving_objs_bbox(plot, [truck_obj_modified], transforms.t_lidar_camera, transforms.t_camera_lidar, color=0xff0000)

plot += k3d.points(np.array([0,0,0]), point_size=0.5, color=0x000000)
plot += k3d.points(lidar_points_cf, point_size=0.04, color=0x000000,shader='mesh')

plot += k3d.points(selected_rp_cf_uncomp, point_size=0.2, color=0x00ff00,shader='mesh')
plot += k3d.points(selected_rp_cf_comp, point_size=0.2, color=0xff1493,shader='mesh')

plot.camera = [
    3.4897, -8.0319, 7.5970,     # position
    4.0071,  1.0298, 9.2022,     # look_at
    -0.0123, -0.2280, 0.9736      # up
]
plot.display()

Output()

In [4]:
snapshot = plot.get_snapshot()
print(snapshot)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

